In [1]:
# =========================
# GPR – Tuning-Pipeline (analog ET/EN/LGBM)
# =========================
# - USE_DYNAMIC_FI_PIPELINE: False => "Full FE"; True => "Dynamic FI"
# - Grid strukturiert wie bei ET/EN/LGBM (inkl. DR/Target-Blocks)
# - Beachte: GPR skaliert ~O(n^3), also Grids moderat halten
# =========================

import os, sys
from pathlib import Path
from itertools import product
import numpy as np
import pandas as pd

# --- 1) Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- 2) Repo-Imports ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,
    EWMA_CORR_SPEC,
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
from src.models.GPR import ForecastModel  # <- der Wrapper oben

# --- 3) Master-Schalter & Meta ---
USE_DYNAMIC_FI_PIPELINE = False
SEED   = 42

MODEL_NAME = "gpr_dynamic_fi" if USE_DYNAMIC_FI_PIPELINE else "gpr"
outputs_for_model(MODEL_NAME)
print(f"Modell {MODEL_NAME} wird getunt.")

# --- 4) Daten laden ---
y    = load_target()           # ΔIP
X_ifo = load_ifo_features()    # für "Full FE"

FI_PATH = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
if USE_DYNAMIC_FI_PIPELINE:
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_PATH)
        rolling_imp   = load_rolling_importance(base_dir=FI_PATH)
        idx = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi = y.loc[idx]
        X_full_lagged = X_full_lagged.loc[idx]
        rolling_imp   = rolling_imp.loc[idx]
        print("Dynamic-FI Daten geladen. Shapes:", X_full_lagged.shape, rolling_imp.shape, y_fi.shape)
    except FileNotFoundError as e:
        print("FEHLER beim Laden der Dynamic-FI Artefakte:", e)
        print("Bitte erst feature_importance.ipynb laufen lassen.")
        raise
else:
    idx = y.index.intersection(X_ifo.index)
    y, X_ifo = y.loc[idx], X_ifo.loc[idx]
    X_full_lagged = rolling_imp = y_fi = None
    print("Full-FE Daten geladen. Shapes:", X_ifo.shape, y.shape)

# --- 5) Base-Config (wie Thesis/ET) ---
def base_cfg_thesis() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis")
    cfg.policy_window   = 24
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 3
    return cfg

cfg0 = base_cfg_thesis()

# --- 6) Corr-Helper ---
def make_corr_spec(kind: str) -> dict:
    if kind == "expanding":
        return dict(DEFAULT_CORR_SPEC)
    elif kind == "ewm":
        return dict(EWMA_CORR_SPEC)
    else:
        raise ValueError("kind must be 'expanding' or 'ewm'")

# --- 7) Grids -------------------------------------------------
if USE_DYNAMIC_FI_PIPELINE:
    # --------- Dynamic FI (nur Modell-HPs + n_features_to_use) ----------
    print("Erstelle HP-Grid für 'Dynamic FI'...")

    N_FEATURES_TO_USE = 20

    kernel_list            = ["rbf", "matern"]
    ard_list               = [True]           # ARD sehr sinnvoll (Pseudo-Importances)
    length_scale_init_list = [0.5]#[0.5, 1.0]
    nu_list                = [1.5]#[1.5, 2.5]       # nur relevant für Matern
    noise_alpha_list       = [1e-4]#[1e-6, 1e-4]
    n_restarts_list        = [0]#[0, 2]
    normalize_y_list       = [True]
    optimizer_list         = ["fmin_l_bfgs_b"]
    weighting_opts         = [{"sample_weight_decay": None}, {"sample_weight_decay": 0.98}]

    def build_model_grid_dynamic_fi():
        grid = []
        for k, ard, ls0, nu, a, nr, norm, opt in product(
            kernel_list, ard_list, length_scale_init_list, nu_list,
            noise_alpha_list, n_restarts_list, normalize_y_list, optimizer_list
        ):
            base = {
                "n_features_to_use": N_FEATURES_TO_USE,
                "kernel": k,
                "ard": ard,
                "length_scale_init": ls0,
                "nu": nu,
                "noise_alpha": a,
                "n_restarts_optimizer": nr,
                "normalize_y": norm,
                "optimizer": opt,
                "seed": SEED,
            }
            for w in weighting_opts:
                hp = dict(base)
                hp.update(w)
                grid.append(hp)
        return grid

    model_grid = build_model_grid_dynamic_fi()

else:
    # ----------------- Full FE (analog ET/EN/LGBM) ---------------------
    print("Erstelle HP-Grid für 'Full FE'...")

    # A) FE/DR-Listen
    corr_options = [
        ("expanding", make_corr_spec("expanding")),
        ("ewm",       make_corr_spec("ewm")),
    ]
    lag_candidates_list   = [(1, 2, 3, 6, 12)]
    top_k_lags_list       = [1]
    use_rm3_list          = [True]
    k1_topk_list          = [100]#[100, 300]
    redundancy_param_list = [0.90]
    dr_options_list       = [
        {"dr_method": "none"},
        #{"dr_method": "pca", "pca_var_target": 0.95, "pca_kmax": 50},
        #{"dr_method": "pls", "pls_components": 8},
    ]

    # B) GPR-HPs
    kernel_list            = ["rbf", "matern"]
    ard_list               = [True, False]
    length_scale_init_list = [0.5]#[0.5, 1.0]
    nu_list                = [1.5]#[1.5, 2.5]  # nur für Matern
    noise_alpha_list       = [1e-4]#[1e-6, 1e-4]
    n_restarts_list        = [0, 2]
    normalize_y_list       = [True]
    optimizer_list         = ["fmin_l_bfgs_b"]

    # C) Target Blocks & Weighting
    target_block_options = [None, ["AR1"], ["Chronos"], ["TSFresh"]]
    weighting_opts       = [{"sample_weight_decay": None}]

    def build_model_grid_full_fe():
        hp_grid = []
        fe_lists = [
            lag_candidates_list,
            top_k_lags_list,
            use_rm3_list,
            k1_topk_list,
            redundancy_param_list,
            dr_options_list,
        ]
        for (corr_tag, corr_spec) in corr_options:
            for (lags, k_lags, rm3, k1, red, dr_opt) in product(*fe_lists):
                if k1 == 100 and dr_opt["dr_method"] != "none":
                    continue
                base_fe = {
                    "lag_candidates": lags,
                    "top_k_lags_per_feature": k_lags,
                    "use_rm3": rm3,
                    "k1_topk": k1,
                    "redundancy_param": red,
                    **dr_opt,
                    "corr_tag": corr_tag,
                    "corr_spec": corr_spec,
                }
                for k, ard, ls0, nu, a, nr, norm, opt in product(
                    kernel_list, ard_list, length_scale_init_list, nu_list,
                    noise_alpha_list, n_restarts_list, normalize_y_list, optimizer_list
                ):
                    base_model = {
                        "kernel": k,
                        "ard": ard,
                        "length_scale_init": ls0,
                        "nu": nu,
                        "noise_alpha": a,
                        "n_restarts_optimizer": nr,
                        "normalize_y": norm,
                        "optimizer": opt,
                        "seed": SEED,
                    }
                    for block_set in target_block_options:
                        for w in weighting_opts:
                            hp = {**base_fe, **base_model, "target_block_set": block_set, **w}
                            hp_grid.append(hp)
        return hp_grid

    model_grid = build_model_grid_full_fe()

print("Optimierte HP-Kombinationen:", len(model_grid))
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid ist leer")

# --- 8) Stage A/B Lauf ---
if model_grid:
    if USE_DYNAMIC_FI_PIPELINE:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,  # Platzhalter
            y=y_fi,
            cfg=cfg0,
            keep_top_k_final=min(10, len(model_grid)),
            min_survivors_per_block=2,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,  # Platzhalter
            y=y_fi,
            cfg=cfg0,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
    else:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,
            y=y,
            cfg=cfg0,
            keep_top_k_final=min(10, len(model_grid)),
            min_survivors_per_block=2,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,
            y=y,
            cfg=cfg0,
        )
else:
    print("Keine gültigen HP-Kombinationen gefunden, Stages übersprungen.")

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Modell gpr wird getunt.
INFO in load_ifo_features: Renaming columns to ensure validity.
Full-FE Daten geladen. Shapes: (407, 2160) (407,)
Erstelle HP-Grid für 'Full FE'...
Optimierte HP-Kombinationen: 64
Erstes HP-Set: {'lag_candidates': (1, 2, 3, 6, 12), 'top_k_lags_per_feature': 1, 'use_rm3': True, 'k1_topk': 100, 'redundancy_param': 0.9, 'dr_method': 'none', 'corr_tag': 'expanding', 'corr_spec': {'mode': 'expanding', 'window': None, 'lam': None}, 'kernel': 'rbf', 'ard': True, 'length_scale_init': 0.5, 'nu': 1.5, 'noise_alpha': 0.0001, 'n_restarts_optimizer': 0, 'normalize_y': True, 'optimizer': 'fmin_l_bfgs_b', 'seed': 42, 'target_block_set': None, 'sample_weight_decay': None}
[Stage A] Using FULL FE pipeline (Gleis 1 & 2).
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=64
  - Config 1/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 pr

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a bette

    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 6/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 7/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 8/64
    · Month 5/20 processed | running...RMSE=1.6407
    · Month 10/20 processed | running...RMSE=1.3090
    · Month 15/20 processed | running...RMSE=1.2294
    · Month 20/20 processed | running...RMSE=1.0966
  - Config 9/64


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a bette

    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 10/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 11/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 12/64
    · Month 5/20 processed | running...RMSE=1.6407
    · Month 10/20 processed | running...RMSE=1.3090
    · Month 15/20 processed | running...RMSE=1.2294
    · Month 20/20 processed | running...RMSE=1.0966
  - Config 13/64


...

    · Month 5/20 processed | running...RMSE=0.8915
    · Month 10/20 processed | running...RMSE=1.2134
    · Month 15/20 processed | running...RMSE=2.6066
    · Month 20/20 processed | running...RMSE=2.6053
  - Config 13/32


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


    · Month 5/20 processed | running...RMSE=0.8915
    · Month 10/20 processed | running...RMSE=1.2134
    · Month 15/20 processed | running...RMSE=2.6066
    · Month 20/20 processed | running...RMSE=2.6053
  - Config 14/32
    · Month 5/20 processed | running...RMSE=0.8915
    · Month 10/20 processed | running...RMSE=1.2134
    · Month 15/20 processed | running...RMSE=2.6066
    · Month 20/20 processed | running...RMSE=2.6053
  - Config 15/32
    · Month 5/20 processed | running...RMSE=0.8917
    · Month 10/20 processed | running...RMSE=1.2129
    · Month 15/20 processed | running...RMSE=2.6029
    · Month 20/20 processed | running...RMSE=2.6025
  - Config 16/32
    · Month 5/20 processed | running...RMSE=0.8917
    · Month 10/20 processed | running...RMSE=1.2129
    · Month 15/20 processed | running...RMSE=2.6029
    · Month 20/20 processed | running...RMSE=2.6025
  - Config 17/32
    · Month 5/20 processed | running...RMSE=0.8915
    · Month 10/20 processed | running...RMSE=1.2134
 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


[Stage B] Month origin t=290 | evaluating 5 configs | active=1
[Stage B] Month origin t=291 | evaluating 5 configs | active=1
[Stage B] Month origin t=292 | evaluating 5 configs | active=1
[Stage B] Month origin t=293 | evaluating 5 configs | active=1
[Stage B] Month origin t=294 | evaluating 5 configs | active=1
[Stage B] Month origin t=295 | evaluating 5 configs | active=1
[Stage B] Month origin t=296 | evaluating 5 configs | active=1
[Stage B] Month origin t=297 | evaluating 5 configs | active=1
[Stage B] Month origin t=298 | evaluating 5 configs | active=1
[Stage B] Month origin t=299 | evaluating 5 configs | active=1
[Stage B] Month origin t=300 | evaluating 5 configs | active=1
[Stage B] Month origin t=301 | evaluating 5 configs | active=1
[Stage B] Month origin t=302 | evaluating 5 configs | active=1
[Stage B] Month origin t=303 | evaluating 5 configs | active=1
[Stage B] Month origin t=304 | evaluating 5 configs | active=1
[Stage B] Month origin t=305 | evaluating 5 configs | a